AMUSE tutorial on high-order bridge
====================

Hierarchical coupling strategies are fundamental parts of AMUSE.
It enables us to combine the output of a wide variety of solvers into a homogeneous solution.
In this example we will be nesting multiple bridges, to show the power of bridge.

We realize this by integrating a star that is orbited by a planet, which is orbited by a moon.
In principle, of course, one can put them all in a single N-body integratory, but here we use bridge.

Let's start by importing the relevant packages.

In [ ]:
import numpy
from amuse.units import (units, constants)

Define a convenient function for obtaining the orbital period of a binary system, and initilize a system with star, planet and moon.

In [ ]:
def orbital_period(Mtot, a):
    return (((4 * numpy.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

def Hill_radius(Mstar, Mplanet, a_planet, e_planet):
    return a_planet*(1.0-e_planet)*(Mplanet/(3*Mstar))**(1./3.)
    
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
def get_star_planet_and_moon():
    Mstar = 1.0|units.MSun
    Mplanet = 1.0|units.MJupiter
    a_planet = 5.2 | units.au
    e_planet = 0.0
    bodies = new_binary_from_orbital_elements(Mstar, Mplanet, a_planet, e_planet,
                                             G=constants.G)
    bodies[0].name = "star"
    bodies[1].name = "planet"
    planet = bodies[bodies.name=="planet"]
    RH_planet = Hill_radius(Mstar, Mplanet, a_planet, e_planet)

    a_moon = 0.1*RH_planet
    e_moon = 0.0
    Mmoon = 0.01*Mplanet
    Pmoon = orbital_period(Mplanet+Mmoon, a_moon)
    moon = new_binary_from_orbital_elements(planet.mass, 
                                            Mmoon, a_moon, e_moon,
                                            G=constants.G)
    moon.position -= moon[0].position
    moon.velocity -= moon[0].velocity
    moon = moon[1].as_set()
    moon.position += planet.position
    moon.velocity += planet.velocity
    moon.semimajor_axis = a_moon
    moon.name = "moon"
    bodies.add_particle(moon)
    bodies.move_to_center()
    return bodies
starplanmoon = get_star_planet_and_moon()
print(starplanmoon)

Now we have the orbits of the three particles, star, planet and moon. to make things a bit more interesting, we make a debris disk around the moon.

In [ ]:
from amuse.ext.protodisk import ProtoPlanetaryDisk

from amuse.lab import nbody_system

def make_disk_around_celestial_body(moon, Mplanet):

    a_moon = moon.semimajor_axis[0]
    e_moon = 0.0
    Mmoon = moon.mass.sum()
    RH_moon = Hill_radius(Mplanet, Mmoon, a_moon, e_moon)
    converter = nbody_system.nbody_to_si(Mmoon, RH_moon)
    Ndisk = 100
    Rin = 0.03
    Rout = 1.0
    Pinner = orbital_period(Mmoon, Rin * a_moon)
    Mdisk = 0.01

    disk = ProtoPlanetaryDisk(Ndisk,
                              convert_nbody=converter,
                              Rmin=Rin,
                              Rmax=Rout,
                              q_out=10.0,
                              discfraction=Mdisk).result
    disk.move_to_center()
    disk.position += moon.position
    disk.velocity += moon.velocity

    disk.mass = Mdisk*Mmoon/float(Ndisk)
    rho = 3.0| (units.g/units.cm**3)
    disk.radius = (disk.mass/(4*rho))**(1./3.)
    disk.name = "disk"
    return disk, Pinner, converter

planet = starplanmoon[starplanmoon.name=="planet"]
moon = starplanmoon[starplanmoon.name=="moon"]
disk, Pinner, converter = make_disk_around_celestial_body(moon, 
                                               planet.mass.sum())

Just define some handy routines for plotting and printing.

In [ ]:
from matplotlib import pyplot
def plot_system(bodies):
    star = bodies[bodies.name=="star"]
    planets = bodies[bodies.name=="planet"]
    moon = bodies[bodies.name=="moon"]
    disk = bodies[bodies.name=="disk"]
    pyplot.scatter(star.x.value_in(units.au), star.y.value_in(units.au), c='y', s=100)
    pyplot.scatter(planet.x.value_in(units.au), planet.y.value_in(units.au), c='b', s=30)
    pyplot.scatter(disk.x.value_in(units.au), disk.y.value_in(units.au), c='k', s=1)
    pyplot.scatter(moon.x.value_in(units.au), moon.y.value_in(units.au), c='r', s=10)
    pyplot.axis("equal")
    pyplot.show()
    
def print_system(bodies):
    star = bodies[bodies.name=="star"]
    planets = bodies[bodies.name=="planet"]
    moon = bodies[bodies.name=="moon"]
    disk = bodies[bodies.name=="disk"]
    print("moon:", moon.position.in_(units.au))
    print("disk:", disk.center_of_mass().in_(units.au))

Declare the various gravity solvers.

In [ ]:
from amuse.community.huayno.interface import Huayno
bodies = starplanmoon
bodies.add_particles(disk)

star = bodies[bodies.name=="star"]
planets = bodies[bodies.name=="planet"]
moon = bodies[bodies.name=="moon"]
disk = bodies[bodies.name=="disk"]
    
gravityA = Huayno(converter)
gravityA.particles.add_particles(star)
gravityA.particles.add_particles(planets)
channel = {"from stars": bodies.new_channel_to(gravityA.particles),
            "to_stars": gravityA.particles.new_channel_to(bodies)}

gravityB = Huayno(converter)
gravityB.particles.add_particles(moon)
gravityB.particles.add_particles(disk)
channel.update({"from_disk": bodies.new_channel_to(gravityB.particles)})
channel.update({"to_disk": gravityB.particles.new_channel_to(bodies)})

plot_system(bodies)

Declare the bi-directional bridge

In [ ]:
from amuse.couple import bridge
from amuse.ext.composition_methods import *
gravity = bridge.Bridge(use_threading=False)
gravity.add_system(gravityA, (gravityB,))
gravity.add_system(gravityB, (gravityA,))
gravity.timestep = 0.1*Pinner

Run the code in the event loop. Store some data and print some diagnostics.

In [ ]:
from amuse.ext.composition_methods import *
from amuse.ext.orbital_elements import orbital_elements_from_binary

def gravity_gravity_bridge(gravityA, gravityB, gravity, bodies,
                         t_end):

    gravity_initial_total_energy = gravityA.get_total_energy() + gravityB.get_total_energy()
    model_time = 0 | units.Myr
    dt = t_end/20.
    
    while model_time < t_end:

        model_time += dt

        orbit_planet = orbital_elements_from_binary(bodies[:2], G=constants.G)
        orbit_moon = orbital_elements_from_binary(bodies[1:3], G=constants.G)
        print("Planet:", "a=", orbit_planet[2].in_(units.AU), "e=", orbit_planet[3])
        print("Moon:", "a=", orbit_moon[2].in_(units.AU), "e=", orbit_moon[3])
        
        dE_gravity = gravity_initial_total_energy/(gravityA.get_total_energy()+gravityB.get_total_energy())
        print("Time:", model_time.in_(units.day), \
              "dE=", dE_gravity)#, dE_hydro

        gravity.evolve_model(model_time)
        channel["to_stars"].copy()
        channel["to_disk"].copy() 
        print_system(bodies)
        print("com=", gravityB.particles.center_of_mass().in_(units.au))
        plot_system(bodies)
    
t_end = 100.0 | units.day
gravity_gravity_bridge(gravityA, gravityB, gravity, 
                       bodies, t_end)
gravity.stop()

You have created a debris around a moon in orbit around a planet that orbits a star.
In principle, such systems have not yet been observed, but we can simulate them anyway.
The code you have been running can be adapted and improved.


Assignments and questions:
---------------

Huayno is usually 2nd order. But we can at least increase the order of the coupling between the codes in the bridge.
This is done using the methods keyword in the bridge operator.
Integrate the system using the SPLIT_4TH_S_M4 or SPLIT_10TH_SS_M35 method (or one of the other methods). These two are 4th and 10th order integration scheme within the bridge.
One advantage of a higher-order scheme is the possibility to increase the time step, and still get a pretty good results. The quality of the result is generally indicated with the degree by which energy is conserved, because we consider energy conservation an essential quality of the reliable numerical solution.
Increasing the order of the integrator then allows us to increase the time step while keeping the error in the energy the same. The integration scheme of the standard bridge is second order. In the above experiment we adopted a time step of 0.001 of the orbital period of the moon. By adopting a higher order bridge coupling we can increase this order.

### Assignment 1:
The plotting routine is somewhat simple. Rewrite the routine
plot_system in such a way that you can study the disk in more detail.
This can be achieved by centering the picture around the planet or on the moon.

Another interesting diagnostic would be the evolution of the moon's orbital parameters.
Make a plot of the moon's semi-major axis and eccentricity in time.

### Assignment 2:

The disk seems to be rather unstable, and the majority of material evaporates within a few orbits.
Adjust the disk in such a way, that it becomes more stable.

### Question 1:

How small do you have to make the disk before it is stable for one orbit?
Does it help to make the time step smaller to acquire a more stable disk?


### Question 2:

The code is not fast.  What part of the code is slow and why?
One way to make the code faster is by increasing the order of the bridge integrator while increasing the time step.
Adopt a 4th order bridge integrator with a time step of 0.01 of the inner orbital period. What effect does this has on the performance and energy conservation when running the code?

Do the same but then with a 10th order integration scheme in the bridge and a bridge time step of 0.1 of the Moon's orbit. What affect that this has on performance and accuracy?

### Assignment 3:

You can improve the code's performance in three ways.

 * One is by running Huayno in parallel. This is done via openMP, not using MPI as most packages in AMUSE. Time the codes' performance, and then run with openMP. 

 * Another way to improve Huayno's performance is by calculating the orbital evolution of the test particles using a Kepler solver rather than integrating the orbit. The bridge will take care of the interaction between disk particles and the planet. The name of the solver is CC_KEPLER, indicating that it adopts a connected-components strategy while solving two-particle interactions using Kepler orbits. 
 
You can change the type of integrator in Huayno using:
gravityB.parameters.inttype_parameter = Huayno.inttypes.CC_KEPLER

 * Eventually, speed-up can be achieved by making the particles in the disk zero-mass. In Huayno such particles are automatically recognized as test particles and not integrated with the N^2 algorithm.

### Question 3:

When adopting a different way of integrating your system, either by making the debris particles zero-mass, or by introducing the CC_KEPLER solver will affect your results. This is noticeable in the morphology of the disk and the number of escaping particles. How can you still validate your results if the outcome depends on the solver?

How much faster do you expect the code is when running with zero-mass disk particles rather than with finite mass?
Why does using openMP not affect your results?

### Assignment 4:

The planetary system we are studying may have a secondary moon. Add a second moon to the planet in a circular orbit at a distance of 0.3 Hill radii around the planet. 

### Question 4:

Where do you put this secondary moon, in the gravityA, in the gravityB or do you bridge with yet another gravity solver?

### Question 5:

What effect does this moon have on the debris disk around the first moon?

### Assignment 5:

Now the star may actually have an equal-mass companion star. You want to keep the orbit of your planet stable. If this secondary star has a circular orbit in the plane of the planet, at what distance should it orbit to prevent it from perturbing the planet?

### Question 6: 

How is the disk around the moon in orbit around the planet affected by the presence of this secondary star?

### Assignment 6:

Debris may fall onto one of the two moons or onto the planet.
Add a stopping condition for collision detection for the star, planet and moons.
Assume all objects to have the same density of 1g/cc.

### Question 7:

Note that it turns out to be tricky to introduce a stopping conditions between two particle sets that live in other codes. How can you solve this problem?

Which of the objects, star, planet or which moon, is most frequenly hit by the debris? 
Can you explain this?